# [Keras + Universal Sentence Encoder = Deep Meter](https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/) 

This notebook creates an autoencoder using the Universal Sentence Encoder. The autoencoder output is CMUdict syllables. The dataset is that subset of Allison Parrish's Project Gutenberg poetry archive which happens to scan in iambic pentameter.

The notebook is based on Chengwei Zhang's example of wrapping the USE inside a larger tensorflow model saves to a Keras model (without save the USE itself in the TF model).

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.

Since there are 10 one-hot values for 10 sets of 6k syllables, this is "multi-label classification"
Changes for multi-label classification:
sigmoid activation instead of softmax
binary_crossentropy

Text format is tab-separated, 2 columns: first text, second multi-level
array of syllables:


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [1]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
%cd /content
!git clone https://github.com/LanceNorskog/deep_meter || true
%cd deep_meter
!git pull
# could not figure out how to read gzipped files as text!
!gunzip -qf blobs/*.gz || true
!gunzip -qf prepped_data/*.gz || true

/content
fatal: destination path 'deep_meter' already exists and is not an empty directory.
/content/deep_meter
Already up to date.
gzip: blobs/*.gz: No such file or directory
gzip: prepped_data/*.gz: No such file or directory


In [2]:
# boilerplate from base notebook
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

Using TensorFlow backend.


In [0]:
# github deep_meter code
import utils
# should not need this to use utils.flatten but is true anyway?
from itertools import chain
import subprocess
import syllables
# misc for this notebook
from ast import literal_eval

import scipy



In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [5]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [0]:
#sentence_labels = utils.flatten([['AE N D'], ['S T AH', 'B ER N'], ['B R AE S'], ['AE N D'], ['T IH N'], ['AE N D'], ['S AA', 'L AH D'], ['G OW L D']])
#other_sentence = utils.flatten([['T UW'], ['M IY'], ['IH T'], ['F L OW Z'], ['AH'], ['S AH', 'L AH N'], ['S T R IY M'], ['AH V'], ['T EH R Z']])
#third_sentence = utils.flatten([['DH AH'], ['N OW', 'B AH L'], ['S IY', 'M AH N'], ['HH UW'], ['W IH TH', 'HH EH L D'], ['DH AH'], ['HH AE N D']])

In [0]:
# read classified poetry lines: text tab [['syll', 'la', 'ble'], ...]
# clip to only most common syllables with syllable manager
def get_data(filename, syll_mgr, num_symbols):
    lines = open(filename, 'r').read().splitlines()
    text_lines = []
    enc_array = []
    num_syllables = syll_mgr.get_size()
    for i in range(0, len(lines)):
      parts = lines[i].split("\t")
      label = utils.flatten(literal_eval(parts[1]))
      if len(label) != num_symbols:
        continue
      enc = np.zeros((num_symbols * syll_mgr.get_size()), dtype=np.int8)
      for j in range(num_symbols):
        enc[num_syllables * j + syll_mgr.get_encoding(label[j])] = 1
      text_lines.append([parts[0]])
      enc_array.append(enc)

    return (np.array(text_lines), np.array(enc_array))


In [0]:
# syllables in descending order of occurrence - 6k in gutenberg.iambic_pentameter, 15k total
# clamp to most common 100 syllables while debugging- use NCE to get all syllables or interesting number
# 98 + pause + wildcard
num_syllables = 1500 
# iambic pentameter
num_symbols = 10
syll_mgr = syllables.syllables(num_syllables)

# slow
num_epochs = 40
adam_0001 = tf.train.AdamOptimizer(0.0005)

In [0]:
(train_text, train_label) = get_data('prepped_data/gutenberg.iambic_pentameter.train', syll_mgr, num_symbols)
num_training = len(train_text)
#train_text = train_text[0:10000]
#train_label = train_label[0:10000]

(test_text, test_label) = get_data('prepped_data/gutenberg.iambic_pentameter.test', syll_mgr, num_symbols)
num_testing = len(test_text)


In [10]:

for i in range(2):
  print(train_text[i][0])
  print(train_label[i])
  syll_mgr.interpret2(train_label[i])

And stubborn brass, and tin, and solid gold;
[0 0 0 ... 0 0 0]
[7, 38, 343, 0, 7, 0, 7, 77, 78, 114]
['AE N D', 'S T AH', 'B ER N', '?', 'AE N D', '?', 'AE N D', 'S AA', 'L AH D', 'G OW L D']
To me, it flows a sullen stream of tears.
[0 0 0 ... 0 0 0]
[30, 59, 231, 0, 36, 175, 937, 0, 40, 977]
['T UW', 'M IY', 'IH T', '?', 'AH', 'S AH', 'L AH N', '?', 'AH V', 'T EH R Z']


In [11]:
messages = []
for i in range(200):
  messages.append(train_text[i][0])
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))
predicts = message_embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
predicts = message_embeddings
print(len(predicts))
first = predicts[0]
for i in range(len(predicts) - 1):
  v = predicts[i]
  #print(scipy.spatial.distance.cosine(v, first))

200


## Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]


## Assemble model

In [14]:
# changed accuracy from 'choose your own accuracy'
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(1024, activation='relu')(embedding)
pred = layers.Dense(num_syllables * num_symbols, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=adam_0001, metrics=['binary_accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 15000)             15375000  
Total params: 15,900,312
Trainable params: 15,900,312
Non-trainable params: 0
_________________________________________________________________


In [15]:
test_label.shape

(4200, 15000)

In [16]:
print(test_text[200])
syll_mgr.interpret2(test_label[200])

['In this: whatever other blunders lie']
[67, 224, 213, 65, 47, 36, 280, 0, 487, 901]
['IH N', 'DH IH S', 'W AH T', 'EH', 'V ER', 'AH', 'DH ER', '?', 'D ER Z', 'L AY']


## Train Keras model and save weights
This only trains and save our Keras layers not the embed module' weights.

In [0]:
use_saved_model=False
retrain=False
if not use_saved_model or not os.path.exists('./model.h5'):
  with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    if retrain:
      model.load_weights('./model.h5')  
    else:
      session.run(tf.tables_initializer())
    history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=num_epochs,
            batch_size=32)
    model.save_weights('./model.h5')


Train on 58498 samples, validate on 4200 samples
Epoch 1/40
58498/58498 [==============================] - 121s 2ms/step - loss: 0.0129 - binary_accuracy: 0.9988 - val_loss: 0.0039 - val_binary_accuracy: 0.9993
Epoch 2/40
58498/58498 [==============================] - 119s 2ms/step - loss: 0.0038 - binary_accuracy: 0.9993 - val_loss: 0.0037 - val_binary_accuracy: 0.9993
Epoch 3/40
58498/58498 [==============================] - 120s 2ms/step - loss: 0.0036 - binary_accuracy: 0.9994 - val_loss: 0.0035 - val_binary_accuracy: 0.9994
Epoch 4/40
58498/58498 [==============================] - 124s 2ms/step - loss: 0.0034 - binary_accuracy: 0.9994 - val_loss: 0.0033 - val_binary_accuracy: 0.9994
Epoch 5/40
58498/58498 [==============================] - 123s 2ms/step - loss: 0.0032 - binary_accuracy: 0.9994 - val_loss: 0.0032 - val_binary_accuracy: 0.9994
Epoch 6/40
58498/58498 [==============================] - 119s 2ms/step - loss: 0.0030 - binary_accuracy: 0.9994 - val_loss: 0.0031 - val_bin

In [59]:
!ls -alh | grep model.h5

-rw-r--r-- 1 root root  61M Oct 24 23:58 model.h5


## Make predictions

In [0]:

#new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(test_text, batch_size=32)

In [61]:
first = predicts[0]
max_d = -1000000
min_d = 1000000
max_i = 0
min_i = 0
for i in range(len(predicts) - 1):
  v = predicts[i]
  d = scipy.spatial.distance.cosine(v, first)
  if max_d < d:
    max_d = d
    max_i = i
  if min_d > d:
    min_d = d
    min_i = i
print("Min and max distances: {0} and {1}".format(min_d, max_d))
print(test_text[max_i])
syll_mgr.interpret2(predicts[max_i])

Min and max distances: 2.5659152136370267e-07 and 0.7241146681791546


In [63]:
for i in range(10,100):
  print(test_text[i])
  print(predicts[i])
  syll_mgr.interpret2(predicts[i])

[52, 86, 86, 0, 86, 0, 86, 0, 36, 0]
['DH AE T', 'IH Z', 'IH Z', '?', 'IH Z', '?', 'IH Z', '?', 'AH', '?']
['Of humble seeming: and, behind them all,']
[4.5766789e-03 3.3007204e-06 1.1491964e-02 ... 4.3276616e-04 4.6846482e-05
 1.3441233e-04]
[7, 0, 7, 0, 7, 0, 7, 0, 7, 0]
['AE N D', '?', 'AE N D', '?', 'AE N D', '?', 'AE N D', '?', 'AE N D', '?']
['Behind his head upon the shoulders lay,']
[1.2936460e-02 2.6487699e-06 2.0789757e-02 ... 1.6308608e-04 6.2689331e-05
 6.8268339e-05]
[36, 0, 10, 0, 7, 0, 10, 0, 10, 0]
['AH', '?', 'HH IH Z', '?', 'AE N D', '?', 'HH IH Z', '?', 'HH IH Z', '?']
['Discover countries, with a kindred heart']
[3.3408054e-03 2.8292488e-06 6.2558122e-02 ... 2.9878257e-04 2.6774514e-05
 1.1755040e-04]
[36, 0, 40, 0, 7, 7, 31, 0, 40, 0]
['AH', '?', 'AH V', '?', 'AE N D', 'AE N D', 'DH AH', '?', 'AH V', '?']
['Was open in his praise, and plaudits rose']
[9.5817139e-03 2.9603564e-06 7.9347733e-03 ... 4.6410994e-04 5.0481260e-05
 1.0920774e-04]
[10, 0, 10, 0, 10, 0, 10,

In [37]:
categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
print("Categorie: {0}".format(categories))
predict_labels = [categories[logit] for logit in predict_logits]
predict_labels

NameError: ignored

In [41]:

os.remove('./model.h5')

FileNotFoundError: ignored